# Concatenate text to create final dataset

In [ ]:
import pandas as pd

In [ ]:
tele_not_sarcastic = pd.read_csv("/content/telegram_not_sarcastic_23k.csv")
random_instances = tele_not_sarcastic.sample(n=1588, random_state=5)
tele_not_sarcastic_left = tele_not_sarcastic.drop(random_instances.index)
random_instances.to_csv('telegram_not_sarcastic_sample.csv')
tele_not_sarcastic_left.to_csv('telegram_not_sarcastic_sample_left.csv')

In [ ]:
telegram = pd.read_csv("telegram_sarcastic.csv")
twitter = pd.read_csv("twitter_sarcastic_not_sarcastic.csv")

In [ ]:
print(telegram.columns)
print(twitter.columns)
print(random_instances.columns)

Index(['Unnamed: 0', 'ConvThread', 'MessageID', 'date', 'UserID', 'Channel',
       'source', 'text', 'is_sarcastic'],
      dtype='object')
Index(['Unnamed: 0', 'is_sarcastic', 'original', 'text', 'username', 'date',
       'source'],
      dtype='object')
Index(['Unnamed: 0', 'MessageID', 'Date', 'Message', 'UserID', 'ChatID'], dtype='object')


In [ ]:
random_instances.rename(columns={"Message":"text"}, inplace=True)
random_instances['source'] = 'Telegram'
random_instances['is_sarcastic'] = 0

In [ ]:
common_cols = ['is_sarcastic', 'text', 'source']
concatenated_df = pd.concat([twitter, telegram, random_instances], axis=0, join='inner', ignore_index=True)

In [ ]:
concatenated_df.isnull().any()

Unnamed: 0      False
is_sarcastic    False
text            False
source          False
dtype: bool

In [ ]:
concatenated_df.to_csv('dataset.csv')

# Cleaning and selecting instances for synthetic data generation

In [ ]:
df = pd.read_csv("/content/dataset.csv", usecols=['source', 'text', 'is_sarcastic'])

In [ ]:
df[df['text'].str.contains('сарказм') == True] # Кожен день водій з пасажиром міняються місцями, и в цьому немає нічого дивного….сарказм

,is_sarcastic,text,source
1542,1,"Взагалі дуже дивно, скільки на ФБ людей, які н...",Twitter
2276,1,"Любиш сарказм? — нєєєєєє, ти шооо, взагалі не ...",Twitter
2583,1,Труднощі життя циніка. Коли намагаєшся бути щи...,Twitter
2756,1,Невже мій сарказм такий вже негарний і відштов...,Twitter
3007,1,"цікаво, як живеться людям, які попереджають пр...",Twitter
3226,1,"- це я мікроекономіку не доганяю, а от сарказм...",Twitter
3569,1,"Хай хоч скільки він герой, добра людина і утют...",Twitter


In [ ]:
# removing sarcasm marker in particular message
df.loc[9065, 'text'] = "Кожен день водій з пасажиром міняються місцями, и в цьому немає нічого дивного...."

In [ ]:
# removing noise instance
df.drop(index=8676, inplace=True) # Ну камон, бро. Ну я ж не можу всюди вішати табличку .

In [ ]:
# balancing examples for 2 classes
index_to_drop = df[df['is_sarcastic'] == False].sample().index
df.drop(index=index_to_drop, inplace=True)

In [ ]:
# finding duplicates
df[df.duplicated(subset=['text']) == True]

,is_sarcastic,text,source
8806,1,Кожна їхня заява краща за попередню,Telegram
8925,1,"Головне не хвилюйся, @orhideous каже що все но...",Telegram
8988,1,Радує що газ і нафта у нас українські,Telegram
9120,1,Ого як швидко СБУ працює\n\n,Telegram


In [ ]:
# droping duplicates
indexes_to_drop = [8806, 8925, 8988, 9120, 10238, 9839]
df.drop(index=indexes_to_drop, inplace=True)

In [ ]:
df[df.duplicated(subset=['text']) == True]

,is_sarcastic,text,source


In [ ]:
print(df[df['is_sarcastic'] == 1].shape[0])
print(df[df['is_sarcastic'] == 0].shape[0])

5378
5380


In [ ]:
# random sampling to balance 2 classes
sarcastic = df[df['is_sarcastic'] == 1]
not_sarcastic = df[df['is_sarcastic'] == 0].sample(5378, random_state=42)

In [ ]:
# uniting two datasets
final_data = pd.concat([sarcastic, not_sarcastic])

In [ ]:
print(final_data[final_data['is_sarcastic'] == 1].shape[0])
print(final_data[final_data['is_sarcastic'] == 0].shape[0])

5378
5378


In [ ]:
final_data.to_csv("dataset_cleaned.csv")

In [ ]:
# random sampling for synthetic sarcastic text generation
sample = final_data[final_data['is_sarcastic'] == 1].sample(n=4)
sample['text'].tolist()

["@PrytSLU що тут сказати, Савченко об'єднує країну!! Так тримати!1",
 '@EspresoTV @Garry51302808 Вона не голосувала за пукіна….',
 "Світу до росіян не існувало, запам'ятайте\n",
 'Я не зрозумію мабуть ніколи, чому комусь лінь шити взуття по стандарту.Чому треба обов‘язково нашити купу шибздиків, і називати їх «МАЛОМЄРКІ»\nНАШО? Це прояв пекла на землі? Шиють взуття агенти диявола? ']